In [ ]:
API_KEY = """"""


import pandas as pd
import os, json, time
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
from openai import OpenAI
from tqdm import tqdm


df = pd.read_csv('filtered.csv')
df = df.sample(n=1, random_state=None)
df



import os, json
from openai import OpenAI

MODEL = "gpt-4o-mini"
client = OpenAI(api_key=API_KEY)

SYSTEM_INSTRUCTIONS = (
    "Rewrite Instagram Reel captions about college admissions into a short, fact-based dialogue between Stewie and Peter Griffin.\n"
    "Format:\n"
    "- Stewie always opens with a sharp question.\n"
    "- The content should be in assumption that the audience is a high school student who is going through the college admissions process and knows a good amount about the process."
    "- Peter provides the majority of the dialogue (≈70–80%), doing detailed fact-based explanations.\n"
    "- Stewie only asks quick questions or gives short affirmations occasionally, without cutting off Peter.\n"
    "- Alternate strictly: Stewie, Peter, Stewie, Peter… until ending.\n"
    "- Keep it to ~1 minute total of talking. That is around 130 words.\n"
    "- End with a short, clear call-to-action (CTA).\n\n"
    "Requirements:\n"
    "- Always include concrete facts when possible: Ivy League admit rates, FAFSA deadlines, SAT/ACT ranges, AP/IB credit, internships, extracurriculars, cost of attendance.\n"
    "- Be clear, accurate, and informative. No vague advice or motivational fluff.\n"
    "- Humor should fit the characters, but facts must stay correct.\n"
    "- Should be formated Stewie: ..., Peter: ..."
)

USER_TEMPLATE = """Turn this Instagram Reel caption into a Stewie–Peter fact-based dialogue.

INPUT:
{raw}
"""


def make_dialogue(transcript: str) -> str:
    """Returns a dense, fact-forward dialogue between Peter & Stewie."""
    resp = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": SYSTEM_INSTRUCTIONS},
            {"role": "user", "content": USER_TEMPLATE.format(raw=transcript.strip())}
        ],
        temperature=0.6,
    )
    return resp.choices[0].message.content.strip()


from tqdm import tqdm
tqdm.pandas()

def generate_transcript(df: pd.DataFrame) -> pd.DataFrame:
    df["dialogue"] = df["caption"].progress_apply(make_dialogue)
    return df




final_script_df = generate_transcript(df)
transcript = final_script_df['dialogue'].item()
print(transcript)

In [ ]:
!pip install requests pydub

import os, io, json, tempfile, requests, pathlib
from typing import List, Tuple, Dict
from pydub import AudioSegment
from google.colab import files  # (left as-is; now unused)

ELEVEN_API_KEY = ""
ELEVEN_STEWIE_VOICE_ID = ""
ELEVEN_PETER_VOICE_ID  = ""

# Tunables
MODEL_ID = "eleven_multilingual_v2"
LINE_GAP_MS = 120  # gap inserted between lines in merged track
VOICE_SETTINGS = {"stability": 0.5, "similarity_boost": 0.75, "style": 0.0, "use_speaker_boost": True}

def _split_dialogue(dialogue_text: str) -> List[Tuple[str, str]]:
    """Return ordered list of (speaker, text) for lines that start with 'Stewie:' or 'Peter:'."""
    ordered = []
    for raw in dialogue_text.splitlines():
        line = raw.strip()
        if not line or line.startswith("["):  # skip [HOOK]/[CTA]
            continue
        lower = line.lower()
        if lower.startswith("stewie:"):
            ordered.append(("Stewie", line.split(":", 1)[1].strip()))
        elif lower.startswith("peter:"):
            ordered.append(("Peter", line.split(":", 1)[1].strip()))
    if not ordered:
        raise ValueError("No dialogue lines detected (expected lines starting with 'Stewie:' or 'Peter:').")
    return ordered

def _tts_elevenlabs(text: str, voice_id: str) -> AudioSegment:
    url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}/stream"
    headers = {"xi-api-key": ELEVEN_API_KEY, "accept": "audio/mpeg", "content-type": "application/json"}
    payload = {"text": text, "model_id": MODEL_ID, "voice_settings": VOICE_SETTINGS, "optimize_streaming_latency": 0}
    r = requests.post(url, headers=headers, data=json.dumps(payload), timeout=60)
    r.raise_for_status()
    return AudioSegment.from_file(io.BytesIO(r.content), format="mp3")

def _format_ts(ms: int) -> str:
    h = ms // 3_600_000; ms %= 3_600_000
    m = ms // 60_000; ms %= 60_000
    s = ms // 1000; ms %= 1000
    return f"{h:02d}:{m:02d}:{s:02d},{ms:03d}"

def _write_srt(cues: List[Tuple[int, int, str]]) -> str:
    """cues: list of (start_ms, end_ms, text)"""
    tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".srt"); tmp.close()
    with open(tmp.name, "w", encoding="utf-8") as f:
        for i, (start, end, text) in enumerate(cues, start=1):
            f.write(f"{i}\n{_format_ts(start)} --> {_format_ts(end)}\n{text}\n\n")
    return tmp.name

def generate_voiceovers_and_srt_elevenlabs(dialogue_text: str, merge: bool = True) -> Dict[str, str]:
    """
    Returns paths:
      - 'merged_path' (mp3): merged alternating track (if merge=True)
      - 'stewie_path' (mp3): concatenated Stewie-only track
      - 'peter_path'  (mp3): concatenated Peter-only track
      - 'srt_path'    (srt): subtitles aligned to merged track (if merge=True)
    """
    ordered = _split_dialogue(dialogue_text)

    # Per-line synthesis for precise timing
    per_line_audio: List[AudioSegment] = []
    per_line_texts: List[str] = []
    for speaker, text in ordered:
        voice_id = ELEVEN_STEWIE_VOICE_ID if speaker == "Stewie" else ELEVEN_PETER_VOICE_ID
        seg = _tts_elevenlabs(text, voice_id)
        per_line_audio.append(seg)
        per_line_texts.append(f"{speaker}: {text}")

    # Build speaker-specific concatenations
    stewie_concat = AudioSegment.silent(duration=0)
    peter_concat  = AudioSegment.silent(duration=0)
    for (speaker, _), seg in zip(ordered, per_line_audio):
        if speaker == "Stewie":
            stewie_concat += seg + AudioSegment.silent(duration=LINE_GAP_MS)
        else:
            peter_concat  += seg + AudioSegment.silent(duration=LINE_GAP_MS)

    # Save individual tracks
    stewie_fd = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3"); stewie_path = stewie_fd.name; stewie_fd.close()
    peter_fd  = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3");  peter_path  = peter_fd.name;  peter_fd.close()
    stewie_concat.export(stewie_path, format="mp3")
    peter_concat.export(peter_path,  format="mp3")

    out = {"stewie_path": stewie_path, "peter_path": peter_path}

    if not merge:
        return out

    # Merge in *spoken order* and build SRT cues from actual durations
    merged = AudioSegment.silent(duration=0)
    cues = []
    cursor = 0
    for seg, text in zip(per_line_audio, per_line_texts):
        start = cursor
        end = start + len(seg)
        cues.append((start, end, text))
        merged += seg
        cursor = end
        # gap between lines (not captioned)
        merged += AudioSegment.silent(duration=LINE_GAP_MS)
        cursor += LINE_GAP_MS

    merged_fd = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3"); merged_path = merged_fd.name; merged_fd.close()
    merged.export(merged_path, format="mp3")
    srt_path = _write_srt(cues)

    out.update({"merged_path": merged_path, "srt_path": srt_path})
    return out


dialogue = transcript

out = generate_voiceovers_and_srt_elevenlabs(dialogue)

# === Changed: keep files on Colab VM instead of downloading to local ===
import shutil
merged_dest = "/content/merged.mp3"
srt_dest = "/content/subtitles.srt"
shutil.copy(out["merged_path"], merged_dest)
shutil.copy(out["srt_path"], srt_dest)
print(f"Saved merged audio to: {merged_dest}")
print(f"Saved subtitles to:    {srt_dest}")

In [ ]:
!apt-get -y install ffmpeg
!pip -q install moviepy pydub


import random
from pathlib import Path
from moviepy.editor import VideoFileClip
from pydub import AudioSegment
from IPython.display import Video, display

def extract_random_clip(input_path: str,
                        audio_path: str,
                        output_path: str = "output.mp4",
                        seed: int | None = None) -> tuple[str, float, float]:
    """
    Take a random chunk of the video whose length equals the duration of audio_path (mp3).
    Guarantees the chunk ends before the video ends.

    Returns: (output_path, start_time_sec, actual_duration_sec)
    """

    input_path = str(input_path)
    output_path = str(output_path)
    if seed is not None:
        random.seed(seed)

    # Get audio duration
    audio = AudioSegment.from_file(audio_path)
    duration = audio.duration_seconds

    # Load video metadata and clip
    with VideoFileClip(input_path) as vid:
        total = float(vid.duration or 0.0)
        if total <= 0:
            raise ValueError("Could not read video duration (is the file valid?)")

        # Clamp if audio is longer than video
        dur = min(duration, total)

        latest_start = max(0.0, total - dur)
        start = 0.0 if latest_start == 0 else random.uniform(0.0, latest_start)
        end = start + dur

        sub = vid.subclip(start, end)
        sub.write_videofile(
            output_path,
            codec="libx264",
            audio_codec="aac",
            temp_audiofile="__temp_aac.m4a",
            remove_temp=True,
            threads=0,
            preset="medium",
            fps=vid.fps or 24
        )

    try:
        from google.colab import files  # type: ignore
        if Path(output_path).exists():
            files.download(output_path)
    except Exception:
        pass

    try:
        display(Video(output_path, embed=True))
    except Exception:
        pass

    return output_path, start, dur

# Example usage: duration will be read from merged.mp3
out, start, dur = extract_random_clip("minecraft.mp4", "merged.mp3", output_path="minecraft_clip.mp4")
print(f"Saved to: {out} | start={start:.3f}s | duration={dur:.3f}s")

In [ ]:
!pip install boto3

import os
import mimetypes
import json
import uuid
import boto3
import requests
from botocore.exceptions import ClientError


R2_ACCOUNT_ID = ""
R2_ACCESS_KEY = ""
R2_SECRET_KEY = ""
R2_BUCKET = ""

CREATOMATE_API_KEY = ""
CREATOMATE_TEMPLATE_ID = ""





# ---------- R2 client (S3-compatible) ----------
r2 = boto3.client(
    "s3",
    aws_access_key_id=R2_ACCESS_KEY,
    aws_secret_access_key=R2_SECRET_KEY,
    endpoint_url=f"https://{R2_ACCOUNT_ID}.r2.cloudflarestorage.com",
    region_name="auto",
)

def _guess_content_type(path: str, default: str = "application/octet-stream") -> str:
    ctype, _ = mimetypes.guess_type(path)
    return ctype or default

def r2_upload_and_presign(local_path: str, key_prefix: str = "inputs/", expires_seconds: int = 3600) -> str:
    """Uploads a local file to R2 (private) and returns a presigned GET URL.
       NOTE: Presigned URLs work on the R2 API hostname, not your custom domain.
    """
    key = f"{key_prefix}{uuid.uuid4().hex}_{os.path.basename(local_path)}"
    content_type = _guess_content_type(local_path)

    # Upload (private by default)
    r2.upload_file(
        Filename=local_path,
        Bucket=R2_BUCKET,
        Key=key,
        ExtraArgs={"ContentType": content_type},
    )

    # Presign GET
    url = r2.generate_presigned_url(
        ClientMethod="get_object",
        Params={"Bucket": R2_BUCKET, "Key": key},
        ExpiresIn=expires_seconds,
    )
    return url

def render_video(mp4_path: str, mp3_path: str) -> dict:
    """Uploads assets to R2, presigns them, and triggers a Creatomate render."""
    # 1) Upload + presign (ensure expiry comfortably exceeds render time)
    mp4_url = r2_upload_and_presign(mp4_path, key_prefix="inputs/video/", expires_seconds=2 * 3600)
    mp3_url = r2_upload_and_presign(mp3_path, key_prefix="inputs/audio/", expires_seconds=2 * 3600)

    # 2) Creatomate render
    url = "https://api.creatomate.com/v2/renders"
    payload = {
        "template_id": CREATOMATE_TEMPLATE_ID,
        "modifications": {
            "Video.source": mp4_url,
            "Audio.source": mp3_url
        }
    }
    resp = requests.post(
        url,
        headers={
            "Authorization": f"Bearer {CREATOMATE_API_KEY}",
            "Content-Type": "application/json",
        },
        data=json.dumps(payload),
        timeout=60,
    )
    resp.raise_for_status()
    return resp.json()


result = render_video("minecraft_clip.mp4", "merged.mp3")
print(result)

# After waiting for video render

In [ ]:
# ✅ One-cell Colab script (Stewie higher/right/bigger; fast fly; no mirroring)
# - Upload in Colab: input.mp4, captions.srt, stewie.png, peter.png
# - Stewie: bottom-left-ish, flies in from LEFT and out to LEFT.
# - Peter:  bottom-right, flies in from RIGHT and out to RIGHT.

# 0) Install deps
!pip -q install moviepy==1.0.3

import re
from pathlib import Path
from IPython.display import Video, display
from moviepy.editor import VideoFileClip, ImageClip, CompositeVideoClip

# 2) Config
VIDEO_PATH    = "createomate.mp4"
SRT_PATH      = "subtitles.srt"
STEWIE_IMG    = "stewie.png"
PETER_IMG     = "peter.png"

MARGIN_PX     = 10          # closer to the edges
STEWIE_H_FRAC = 0.32        # Stewie a tiny bit bigger
PETER_H_FRAC  = 0.44        # Peter size relative to video height
FLY_DUR       = 0.06        # faster fly-in and fly-out
OVERSHOOT_PX  = 24          # larger offscreen buffer for crisp exits

# Stewie custom offsets
STEWIE_EXTRA_RIGHT = 10     # px farther right
STEWIE_EXTRA_UP    = 0     # px higher

# 3) Parse SRT for speaker intervals
time_pat = re.compile(
    r"(\d{2}):(\d{2}):(\d{2}),(\d{3})\s*-->\s*(\d{2}):(\d{2}):(\d{2}),(\d{3})"
)
def _sec(h,m,s,ms): return int(h)*3600 + int(m)*60 + int(s) + int(ms)/1000.0

def parse_srt_for_speakers(srt_path):
    with open(srt_path, "r", encoding="utf-8") as f:
        blob = f.read()
    blocks = re.split(r"\n\s*\n", blob.strip())
    out = []
    for b in blocks:
        lines = [ln for ln in b.splitlines() if ln.strip()]
        if not lines: continue
        mt = time_pat.search(lines[1]) if len(lines) > 1 else None
        text_idx = 2 if mt else 1
        if not mt:
            mt = time_pat.search(lines[0])
        if not mt: continue
        sh,sm,ss,sms, eh,em,es,ems = mt.groups()
        start, end = _sec(sh,sm,ss,sms), _sec(eh,em,es,ems)
        if end <= start: continue
        text = "\n".join(lines[text_idx:]).strip()
        speaker = None
        if re.match(r"^\s*stewie\s*:", text, flags=re.I): speaker = "stewie"
        elif re.match(r"^\s*peter\s*:", text, flags=re.I): speaker = "peter"
        if speaker: out.append({"start": start, "end": end, "speaker": speaker})
    return sorted(out, key=lambda e: e["start"])

# 4) Build composite with fly animations
assert Path(VIDEO_PATH).exists(), "createomate.mp4 not found"
assert Path(SRT_PATH).exists(), "subtitles.srt not found"
assert Path(STEWIE_IMG).exists(), "stewie.png not found"
assert Path(PETER_IMG).exists(), "peter.png not found"

vid = VideoFileClip(VIDEO_PATH)
vw, vh = vid.w, vid.h

# Prepare base sprites
stewie_base = ImageClip(STEWIE_IMG).resize(height=vh * STEWIE_H_FRAC)
peter_base  = ImageClip(PETER_IMG).resize(height=vh * PETER_H_FRAC)

# Resting positions
stewie_y = vh - stewie_base.h - MARGIN_PX
peter_y  = vh - peter_base.h  - MARGIN_PX + 10
stewie_on_x = MARGIN_PX + STEWIE_EXTRA_RIGHT
peter_on_x  = vw - peter_base.w - MARGIN_PX

# Offscreen positions
stewie_off_x = -stewie_base.w - OVERSHOOT_PX
peter_off_x  = vw + OVERSHOOT_PX

def smoothstep(u): return max(0.0, min(1.0, u*u*(3 - 2*u)))
def lerp(a, b, u): return a + (b - a) * u

def make_fly_clip(sprite, start, end, side):
    dur = max(0.001, end - start)
    t_in  = min(FLY_DUR, dur/2.0)
    t_out = min(FLY_DUR, dur/2.0)
    hold_start = t_in
    hold_end   = max(hold_start, dur - t_out)

    if side == "left":
        x_off, x_on, y = stewie_off_x, stewie_on_x, stewie_y
    else:
        x_off, x_on, y = peter_off_x,  peter_on_x,  peter_y

    def pos_func(t):
        if t <= hold_start:
            u = smoothstep(t / t_in) if t_in > 0 else 1.0
            x = lerp(x_off, x_on, u)
        elif t < hold_end:
            x = x_on
        else:
            u = smoothstep((t - hold_end) / max(1e-6, (dur - hold_end)))
            x = lerp(x_on, x_off, u)
        return (x, y)

    return sprite.set_start(start).set_duration(dur).set_pos(pos_func)

events = parse_srt_for_speakers(SRT_PATH)

clips = [vid]
for ev in events:
    if ev["speaker"] == "stewie":
        clips.append(make_fly_clip(stewie_base, ev["start"], ev["end"], side="left"))
    elif ev["speaker"] == "peter":
        clips.append(make_fly_clip(peter_base,  ev["start"], ev["end"], side="right"))

final = CompositeVideoClip(clips, size=vid.size)

# 5) Render and preview
out_name = "output_with_characters.mp4"
final.write_videofile(
    out_name,
    codec="libx264",
    audio_codec="aac",
    fps=vid.fps or 30,
    preset="medium",
    threads=4
)

display(Video(out_name, embed=True, html_attributes="controls loop muted playsinline"))

try:
    from google.colab import files  # type: ignore
    files.download(out_name)
except Exception:
    pass


In [ ]:
!pip -q install moviepy==1.0.3 srt==3.5.3 pillow==10.4.0 icrawler pandas==2.2.2 openai==1.40.6

import os, re, io, random, string, tempfile
import pandas as pd
import srt
from pathlib import Path
from datetime import timedelta
from PIL import Image
from icrawler.builtin import GoogleImageCrawler
from moviepy.editor import VideoFileClip, ImageClip, CompositeVideoClip, vfx

# -----------------------------
# Config
# -----------------------------
API_KEY = """"""
OPENAI_MODEL = "gpt-4o-mini"

VIDEO_PATH = "output_with_characters.mp4"
SRT_PATH   = "subtitles.srt"
OUT_DIR = Path("downloads")
MANIFEST_CSV = "overlay_manifest.csv"
OUT_VIDEO = "final_output.mp4"

assert os.path.exists(VIDEO_PATH), f"Missing {VIDEO_PATH}"
assert os.path.exists(SRT_PATH),   f"Missing {SRT_PATH}"

# -----------------------------
# Helpers
# -----------------------------
def read_srt(path: str):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        return list(srt.parse(f.read()))

def to_seconds(td: timedelta) -> float:
    return td.total_seconds()

def sanitize_seed(text: str) -> str:
    t = (text or "").strip()
    t = re.sub(r"^[A-Za-z .'\-]{2,24}:\s+", "", t)
    t = re.sub(r"[\(\[][^)\]]*[\)\]]", "", t)
    t = re.sub(r"(https?://\S+)|(@\w+)|(#\w+)", "", t)
    t = re.sub(r"\s+", " ", t).strip()
    words = t.split()
    return " ".join(words[:12])

def slugify(s: str, maxlen: int = 60) -> str:
    s = re.sub(r"[^A-Za-z0-9._\- ]+", "", s).strip().lower()
    s = re.sub(r"\s+", "-", s)
    if len(s) > maxlen:
        s = s[:maxlen].rstrip("-")
    return s or "q"

# -----------------------------
# OpenAI (optional) — broad college visuals (icons, campuses, forms, essays, flowcharts)
# -----------------------------
def refine_query(text: str) -> str:
    """
    Turn a subtitle into a <= 8-word, college-admissions-focused image query.
    Accept logos, icons, campus/app screenshots, rankings/scholarships, SAT/ACT/GPA visuals,
    essay prompts/supplementals/example essays, and flowcharts/diagrams/guides (College Essay Guy style).
    """
    base = sanitize_seed(text)
    if not base:
        base = "college admissions"
    if not API_KEY:
        return f"{base} college essay flowchart"
    try:
        from openai import OpenAI
        client = OpenAI(api_key=API_KEY)
        system = (
            "Convert the caption into a concise (<= 8 words) image-search query "
            "for COLLEGE ADMISSIONS visuals. Accept: logos/icons (Common App, FAFSA, Ivy League, Stanford, MIT), "
            "campus photos, application screenshots, clubs/competitions, US News rankings, scholarships, SAT/ACT/GPA visuals, "
            "essay prompts, supplementals, example essays, and flowcharts/diagrams/guides (College Essay Guy style). "
            "Avoid vague phrasing. Output ONLY the query."
        )
        user = f"Caption: {base}\nReturn a short, college admissions–related image search query."
        resp = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[{"role":"system","content":system},{"role":"user","content":user}],
            temperature=0.2,
            max_tokens=24,
        )
        q = (resp.choices[0].message.content or "").strip()
        q = re.sub(r"[\n\r]+", " ", q).strip()
        return q or f"{base} college essay flowchart"
    except Exception:
        return f"{base} college essay flowchart"

# -----------------------------
# Image crawling (broad to include diagrams/flowcharts/docs)
# -----------------------------
def crawl_one_image(query: str, dest_dir: Path, max_num: int = 1):
    dest_dir.mkdir(parents=True, exist_ok=True)
    existing = [p for p in dest_dir.glob("*") if p.is_file()]
    if existing:
        return [str(p) for p in existing]
    crawler = GoogleImageCrawler(storage={'root_dir': str(dest_dir)})
    try:
        crawler.crawl(keyword=query, max_num=max_num, filters=None)
    except Exception as e:
        print(f"[WARN] Crawl failed for '{query}': {e}")
        return []
    files = [str(p) for p in dest_dir.glob("*") if p.is_file()]
    return files

def add_soft_border(img: Image.Image, pad: int = 10, alpha_bg: int = 220) -> Image.Image:
    if img.mode != "RGBA":
        img = img.convert("RGBA")
    w, h = img.size
    bg = Image.new("RGBA", (w + 2*pad, h + 2*pad), (255, 255, 255, alpha_bg))
    bg.paste(img, (pad, pad), img)
    return bg

# -----------------------------
# 1) Build refined queries
# -----------------------------
subs = read_srt(SRT_PATH)
rows = []
unique_queries = {}

for idx, sub in enumerate(subs, start=1):
    start_s = to_seconds(sub.start)
    end_s   = to_seconds(sub.end)
    if end_s <= start_s:
        continue
    refined = refine_query(sub.content).strip()
    q_key = refined.lower()
    if q_key not in unique_queries:
        unique_queries[q_key] = refined
    rows.append({
        "line_index": idx,
        "start_time": start_s,
        "end_time": end_s,
        "text": sub.content.strip(),
        "query": refined,
        "image_path": ""
    })

print(f"Subtitle lines parsed: {len(rows)} | Unique refined queries: {len(unique_queries)}")

# -----------------------------
# 2) Crawl images for all unique queries FIRST
# -----------------------------
OUT_DIR.mkdir(parents=True, exist_ok=True)
query_to_imagepath = {}

for q_key, q in unique_queries.items():
    folder = OUT_DIR / slugify(q)
    files = crawl_one_image(q, folder, max_num=1)
    if files:
        query_to_imagepath[q] = files[0]
        print(f"[OK] {q} -> {files[0]}")
    else:
        query_to_imagepath[q] = ""
        print(f"[MISS] {q} -> (no image)")

# Map back to rows + write manifest
for r in rows:
    r["image_path"] = query_to_imagepath.get(r["query"], "")
pd.DataFrame(rows).to_csv(MANIFEST_CSV, index=False, encoding="utf-8")
print(f"Manifest: {Path(MANIFEST_CSV).resolve()}")

# -----------------------------
# 3) Build overlay clips: 85% width, top-centered, never below 40% height
# -----------------------------
video = VideoFileClip(VIDEO_PATH)
video_w, video_h = video.w, video.h

TARGET_W_FRAC = 0.85                   # <- 85% of video width
MIN_IMG_W = max(140, int(video_w * 0.10))
MAX_H = int(video_h * 0.40)            # <- images must not extend below 40% of frame height

# Transitions (fast)
FADE = 0.05
POP  = 0.06
SCALE_MIN = 0.60

# Top-centered baseline
X_ANCHOR = "center"
TOP_MARGIN = int(video_h * 0.08)       # try to keep near the top

overlay_clips = []

def prepare_png_for_overlay(img_path: str):
    """
    Resize to 85% width, crop so total (with border) <= MAX_H,
    add border, return (temp_png_path, (w, h)) of final image.
    """
    im = Image.open(img_path).convert("RGBA")

    # 1) Resize by width target (85% of video width)
    target_w = max(MIN_IMG_W, int(video_w * TARGET_W_FRAC))
    scale = target_w / im.width
    new_w, new_h = target_w, int(im.height * scale)
    im = im.resize((new_w, new_h), Image.LANCZOS)

    # 2) If too tall, center-crop vertically to fit within MAX_H - pad allowance
    pad_each = 10
    allowable_h_before_pad = MAX_H - 2*pad_each
    if im.height > allowable_h_before_pad:
        crop_h = max(10, allowable_h_before_pad)
        top = max(0, (im.height - crop_h)//2)
        im = im.crop((0, top, im.width, top + crop_h))

    # 3) Add soft border
    im = add_soft_border(im, pad=pad_each, alpha_bg=220)

    # 4) Enforce MAX_H again (in case pad pushed it over)
    if im.height > MAX_H:
        top = max(0, (im.height - MAX_H)//2)
        im = im.crop((0, top, im.width, top + MAX_H))

    tmp = os.path.join(tempfile.gettempdir(), "ovl_" + "".join(random.choices(string.ascii_lowercase+string.digits, k=8)) + ".png")
    im.save(tmp, "PNG")
    return tmp, im.size  # (w, h)

for r in rows:
    img_path = r["image_path"]
    if not img_path or not os.path.exists(img_path):
        continue

    start_t = r["start_time"]
    end_t   = r["end_time"]
    dur = end_t - start_t
    if dur <= 0.03:
        continue

    png_path, (ow, oh) = prepare_png_for_overlay(img_path)

    # Position: centered horizontally, and as high as possible WITHOUT crossing 40% height.
    # Ensure top_y + overlay_height <= MAX_H (i.e., bottom edge no lower than 40% of frame).
    allowed_top = max(0, MAX_H - oh)     # highest y so that bottom == MAX_H
    pos_y = min(TOP_MARGIN, allowed_top) # keep toward top but honor the 40% ceiling
    pos = (X_ANCHOR, pos_y)

    clip = (ImageClip(png_path)
            .set_start(start_t)
            .set_duration(dur)
            .set_position(pos)
            .set_opacity(1.0))

    # Ultra-fast pop from center + quick fade
    D = clip.duration
    def scale_fun(t, D=D, P=POP, s0=SCALE_MIN):
        if t < P:
            return s0 + (1.0 - s0) * (t / P)
        elif t > D - P:
            return s0 + (1.0 - s0) * ((D - t) / P)
        else:
            return 1.0

    clip = (clip
            .fx(vfx.resize, scale_fun)
            .fx(vfx.fadein, FADE)
            .fx(vfx.fadeout, FADE))

    overlay_clips.append(clip)

print(f"Overlay clips built: {len(overlay_clips)}")

# -----------------------------
# 4) Render final video
# -----------------------------
final = CompositeVideoClip([video] + overlay_clips) if overlay_clips else video
fps = getattr(video, "fps", 24) or 24

final.write_videofile(
    OUT_VIDEO,
    codec="libx264",
    audio_codec="aac",
    fps=fps,
    threads=4,
    preset="medium",
    logger=None
)

print(f"Done. Wrote: {OUT_VIDEO}")
try:
    from IPython.display import Video, display
    display(Video(OUT_VIDEO, embed=True))
except Exception:
    pass